##Make necessary imports

In [1]:
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
import os
from xlrd import *
import csv 
import sys
import win32com.client as w3c
from tempfile import NamedTemporaryFile

In [3]:
PATH = 'C:/Users/smsoh/Documents/GitHub/CLEF2021/task3/data/Task3a_Batch_1.xlsx'
PASS = 'fake@2021'

In [4]:
xlapp = w3c.Dispatch("Excel.Application")
xlwb = xlapp.Workbooks.Open(PATH, False, True, None, PASS)

In [5]:
#df = pd.read_csv("data/Task3a_Batch_1.xlsx")
f = NamedTemporaryFile(delete=False, suffix='.csv')
f.close()
os.unlink(f.name)

xlCSVWindows = 0x17
xlwb.SaveAs(Filename=f.name, FileFormat=xlCSVWindows)
df = pd.read_csv(f.name)
df

,public_id,text,title,our rating
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,partially false
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,partially false
3,f14e8eb6,But things took a turn for the worse when riot...,Obama�s Daughters Caught on Camera Burning US ...,false
4,faf024d6,It�s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff�s 78 Visits ...,false
...,...,...,...,...
432,e01f532a,Rosemary Frei and Patrick Corbett A high-prof...,�No one has died from the coronavirus�,false
433,3c791c29,As the coronavirus pandemic continues to devas...,Why Did 21 Million Phone Numbers Disappear fro...,false
434,fe6a9262,Belgium Health Minister Maggie de Block has pu...,Belgium Health Minister puts ban on non-essent...,false
435,97113f55,"Responding To Backlash, Netflix Clarifies Its ...","Responding To Backlash, Netflix Clarifies Its ...",false


In [6]:
df.head()

,public_id,text,title,our rating
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,partially false
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,partially false
3,f14e8eb6,But things took a turn for the worse when riot...,Obama�s Daughters Caught on Camera Burning US ...,false
4,faf024d6,It�s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff�s 78 Visits ...,false


In [7]:
labels=df['our rating']
labels.head()

0              false
1    partially false
2    partially false
3              false
4              false
Name: our rating, dtype: object

New update

In [ ]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [ ]:
df_true = pd.read_csv("data/True.csv")
df_true = df_true.drop([ "title","subject","date"], axis = 1)
df_true['class'] = 1
df_fake = pd.read_csv("data/Fake.csv")
df_fake = df_fake.drop([ "title","subject","date"], axis = 1)
df_fake['class'] = 0

df = pd.concat([df_fake, df_true], axis=0)
df.sort_index(inplace=True)
df["text"] = df_t["text"].apply(wordopt)
labels=df['class']
df

#DataFlair - Split the dataset

In [8]:
x_train,x_test,y_train,y_test=train_test_split(df['text'], labels, test_size=0.3, random_state=8)

#DataFlair - Initialize a TfidfVectorizer

In [9]:
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)

#DataFlair - Fit and transform train set, transform test set

In [10]:
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

#DataFlair - Initialize a PassiveAggressiveClassifier

In [11]:
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)

PassiveAggressiveClassifier(max_iter=50)

#DataFlair - Predict on the test set and calculate accuracy

In [12]:
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 76.52%


#DataFlair - Build confusion matrix

In [13]:
confusion_matrix(y_test,y_pred, labels=['FALSE','TRUE','partially false'])

array([[ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0, 15]], dtype=int64)